## Welcome to the National Water Model (NWM) Sandbox 2! 

<strong><em>Created by <a href="https://www.linkedin.com/in/justin-hunter-0b86871a6/" target="_blank">Justin Hunter</a>, <a href="https://www.linkedin.com/in/danames/" target="_blank">Dr. Dan Ames</a>, and <a href="https://www.linkedin.com/in/easton-perkins-02968a156/" target="_blank">Easton Perkins</a>.</em></strong><br>
<em><strong>May, 2021. Brigham Young University. Provo, Utah.
<a href="hydroinformatics.byu.edu" target="_blank">BYU Hydroinformatics Lab</a>.</em></strong>

In this Jupyter Notebook we will play around with a NetCDF (Network Common Data Form) file obtained from the NWM using a Python package called xarray. NetCDF files are a standardized way of exchanging scientific data. NetCDF is well suited for multidimensional datasets containing meteorological or observational data. NetCDF files work well for NWM forecasts and contain a lot of useful metadata. xarray is a Python package which uses NumPy and pandas and works well with NetCDF. 

<em><h4>Imports</h4></em>

The next cell installs or imports some Python modules or packages that will be used in this notebook. A brief explanation of what each one does is included below:
* xarray makes it easier to work with multidimensional datasets like the NWM forecasts. 
* NumPy is a useful package for working with data arrays.
* Importing the date type from the datetime module allows us to call todays date. 
* The os module allows us to communicate with the operating system. 
* The requests module lets us make requests to web pages. 
* pandas is a useful library for data manipulation and analysis.
* Folium is going to help us create a map at the end of the notebook.
* matplotlib will help us create plots.

In [ ]:
import xarray as xr
import numpy as np
from datetime import date
import os
import requests
import pandas as pd
# Install folium
!pip install folium -q
# Import folium
import folium
%matplotlib inline
import matplotlib.pyplot as plt

<em><h4>Functions</em></h4>

This next cell should look familiar! It uses two of the functions that were in the first sandbox. These functions build the appropriate url for a NWM forecast and the get the NetCDF file stored at that url.

In [ ]:
# Get todays date
today = date.today()
today = str(today)
today = today.replace("-", "")

# This function gets the desired forecasts file name.
def GetForecastFileName(ForecastStart='00', Type = 'short_range', Member='1', TimeStep = '001'):
  BaseName = 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/nwm/prod/nwm.'

  if (Type == 'short_range'):
    return BaseName + today + '/short_range/nwm.t' + ForecastStart +'z.short_range.channel_rt.f' + TimeStep + '.conus.nc'
  elif (Type == 'medium_range'): 
    return BaseName + today + '/medium_range_mem' + Member + '/nwm.t' + ForecastStart +'z.medium_range.channel_rt_' + Member + '.f' + TimeStep + '.conus.nc'
  elif (Type == 'long_range'):
    return BaseName + today + '/long_range_mem' + Member + '/nwm.t' + ForecastStart +'z.long_range.channel_rt_' + Member + '.f' + TimeStep + '.conus.nc'
  else:
    return 'error'

# This function gets the actual forecast file
def GetForecastFile(Url = 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/nwm/prod/nwm.20210321/short_range/nwm.t00z.short_range.channel_rt.f001.conus.nc'):
  FileName = os.path.basename(Url)
  if os.path.exists(FileName):
    os.remove(FileName)
  r = requests.get(Url, allow_redirects=True)
  open(FileName, 'wb').write(r.content)
  return FileName

<em><h4>Variables</em></h4>

Next let's define some variables that we can use to download a forecast of our choice. We get the url and then the file and store it in a variable called 'path'.

In [ ]:
# These variables should look familiar to you from the first sandbox
ForecastStart = '00'
Type = 'short_range'
Member = '1'
Timestep = '01'

url = GetForecastFileName(ForecastStart,Type,Member,Timestep)
path = GetForecastFile(url)

The next cell creates a variable called 'ds' and opens our downloaded NetCDF file as an xarray dataset. You'll be able to see the different dataset dimensions, coordinates, variables, and attributes after running this cell. Look around and try to learn a little bit about the metadata included with this dataset.

<em><h4>xarray</em></h4>

In [ ]:
ds = xr.open_dataset(path)
ds

<em><h4>Plot</em></h4>

Now let's try to visualize this dataset using matplotlib like we've done before. 

In [ ]:
plt.rcParams['figure.figsize'] = (8,6)
ds.streamflow.plot()

That plot isn't great. Because each NetCDF file contains a certain forecast (short, medium, long, etc.) at a certain time for the entire NWM network (every single reach covered by the NWM), the plot is displaying Reach IDs on the x axis and streamflow on the y. This isn't very helpful. Let's see if we can find a better way to visualize these forecasts.

<em><h4>Map</em></h4>

First we should define some variables to pass to folium in order to create our map.

In [ ]:
#zoom level
zoom = 4
#lat and long coordinates for the center of the map
location = (39,-96)
#Map frame height
height = 500
#points to where the river reach layers are stored
url_stem = 'https://geoserver.hydroshare.org/geoserver/wms'
#Name of the layer that will appear on the map
name = 'NWM River Reaches'
#Gives the layer an appropriate format on the map
formt = 'image/png'
#Makes the layer opaque so that the basemap can still be seen
opacity = .5

Next, we need to create our map with the folium.Map method and then add a layer to it called NWM River Reaches. That layer consists of two shapefiles which are being pulled into the map via web mapping services (wms) from a Hydroshare resource where they are published. The Hydroshare resource id is 'HS-151be7ba211f4de8abbc4a64795df682' and the two shapefiles are respectively named 'NWMReaches1' and 'NWMReaches2'.

In [ ]:
# Create map
fol_map = folium.Map(location=location, zoom_start=zoom, height=height)

# Create layer from wms 
layer = folium.raster_layers.WmsTileLayer(
    url_stem,
    name=name,
    layers=(f'HS-151be7ba211f4de8abbc4a64795df682:NWMReaches1',f'HS-151be7ba211f4de8abbc4a64795df682:NWMReaches2'),
    wms_format=formt,
    transparent=True,
    opacity=opacity
)

# Add layer to map
layer.add_to(fol_map)

Now we can display the map. The first line of code in the next cell also adds layer control to the map which allows us to turn the NWM River Reaches layer off and on again via a button on the map itself. Zoom controls are automatically included with the map. 

In [ ]:
# Add layer control
folium.LayerControl().add_to(fol_map)

# Display map
display(fol_map)

This map shows all of the NWM reaches, but we haven't actually done anything which would allow us to view the forecasts yet.